# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

THƯ VIỆN

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

TẢI TẬP DỮ LIỆU PARKINSONS

In [ ]:
path = '..\data\parkinsons.data'
data = pd.read_csv(path, sep=',')
data.head()

THUỘC TÍNH(KHÔNG BAO GỒM CỘT NAME) VÀ NHÃN(CỘT STATUS) CỦA TẬP DỮ LIỆU

In [ ]:
data.info()

TÁCH THUỘC TÍNH VÀ NHÃN

In [ ]:
X = data.drop(columns=['name', 'status'])
y = data['status']

In [ ]:
X.head()

KIỂM TRA DỮ LIỆU KHUYẾT THIẾU

In [ ]:
X.isnull().sum()

In [ ]:
y.head()

KIỂM TRA ĐỘ CÂN BẰNG NHÃN

In [ ]:
y.value_counts()

HOLD OUT

In [ ]:
# Thử từng k để chọn k tốt nhất.
# Chọn k lẻ vì bài toán có 2 lớp, tránh hòa phiếu
k_values = range(3,22, 2)

result = {}
num_iterations = 100
# lặp 100 lần mỗi k
sss = StratifiedShuffleSplit(n_splits=num_iterations, test_size=0.2, random_state=42)
for k in k_values:
    f1_scores = []

    for split_idx, (train_idx, test_idx) in enumerate(sss.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        scaler = StandardScaler()
        X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index,columns=X_train.columns)
        X_test_scaled = pd.DataFrame(scaler.transform(X_test),index=X_test.index,columns=X_test.columns)
        # StandardScaler chuẩn hóa theo công thức Z-score:
        #     X_scaled = (X - μ) / σ
        # Trong đó:
        #     - X: giá trị gốc
        #     - μ: trung bình của thuộc tính (mean)
        #     - σ: độ lệch chuẩn (standard deviation)
        # Kết quả: dữ liệu có mean = 0 và std = 1

        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_resampled, y_resampled)
        y_pred = model.predict(X_test_scaled)

        f1 = f1_score(y_test, y_pred, average='binary')
        f1_scores.append(f1)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    result[k] = mean_f1, std_f1

# Tìm k tốt nhất theo F1-score trung bình
best_k_value, (best_mean_f1, best_std_f1) = max(result.items(), key=lambda x: x[1][0])

SO SÁNH F1 TRUNG BÌNH VÀ ĐỘ LỆCH CHUẨN CỦA MÔ HÌNH KNN THEO K

In [ ]:
k_vals = list(result.keys())
means = [result[k][0] * 100 for k in k_vals]   # F1-score trung bình của từng lần lặp k
stds = [result[k][1] * 100 for k in k_vals]   # Độ lệch chuẩn F1 (%) của từng lần lặp k

plt.figure(figsize=(10, 6))
plt.errorbar(k_vals, means, yerr=stds, fmt='-o', capsize=4, color='darkgreen')
plt.axvline(best_k_value, color='red', linestyle='--', label=f'Best k = {best_k_value}')
plt.xlabel('Giá trị k')
plt.xticks(range(min(k_vals), max(k_vals)+1, 2))
plt.yticks(range(76, 101, 2))
plt.ylabel('F1-score trung bình (%)')
plt.title('F1-score theo k và độ ổn định (± std)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


ĐỘ CHÍNH XÁC CỦA MÔ HÌNH

In [ ]:
print(f"F1-score (%) = {best_mean_f1 * 100:.2f}%")

In [ ]:
print("k |   F1 (%)|   Std   |  RSD (%)")
print("-" * 32)
for k, (mean, std) in result.items():
    rsd = (std / mean) * 100
    print(f"{k:2} | {mean * 100:.2f}%  | {std:.4f} | {rsd:.2f}%")